# AIG230 NLP (Week 3 Lab) — Notebook 2: Statistical Language Models (Train, Test, Evaluate)

This notebook focuses on **n-gram Statistical Language Models (SLMs)**:
- Train **unigram**, **bigram**, **trigram** models
- Handle **OOV** with `<UNK>`
- Apply **smoothing** (Add-k)
- Evaluate with **cross-entropy** and **perplexity**
- Do **next-word prediction** and simple **text generation**

> Industry framing: even if modern systems use neural LMs, n-gram LMs are still useful for
baselines, constrained domains, and for understanding evaluation.


## 0) Setup


In [1]:
!pip install numpy pandas

In [2]:

import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


## 1) Data: domain text you might see in real systems


We use short texts that resemble:
- release notes
- incident summaries
- operational runbooks
- customer support messaging

In practice, you would load thousands to millions of lines.


In [3]:

corpus = [
    "user cannot login after password change",
    "vpn connection drops during peak hours",
    "email attachments not downloading properly",
    "mfa code expired before user could enter it",
    "system performance slow after recent update",
    "printer not responding after driver update",
    "shared drive access denied for new employees",
    "wifi signal weak in conference room",
    "application crashes when uploading files",
    "api authentication token expired unexpectedly",
    "mailbox storage quota exceeded",
    "remote desktop disconnects randomly",
    "user account locked after multiple attempts",
    "software installation fails due to permissions",
    "network latency causing timeout errors",
    "calendar events not syncing across devices",
    "mobile app notifications delayed",
    "database backup failed overnight",
    "file upload stuck at ninety percent",
    "password reset email not received"
]

# Train/test split at sentence level
random.seed(42)
random.shuffle(corpus)
split = int(0.75 * len(corpus))
train_texts = corpus[:split]
test_texts = corpus[split:]

len(train_texts), len(test_texts), train_texts[:2], test_texts[:2]


(15,
 5,
 ['password reset email not received',
  'printer not responding after driver update'],
 ['mobile app notifications delayed', 'wifi signal weak in conference room'])

## 2) Tokenization + special tokens


We will:
- lowercase
- keep alphanumerics
- split on whitespace
- add sentence boundary tokens: `<s>` and `</s>`

We will also map rare tokens to `<UNK>` based on training frequency.


In [4]:

def tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

def add_boundaries(tokens: List[str], n: int) -> List[str]:
    # For n-grams, prepend (n-1) start tokens for simpler context handling
    return ["<s>"]*(n-1) + tokens + ["</s>"]

# Example
tokens = tokenize("Printer driver install fails with error 1603")
add_boundaries(tokens, n=3)


['<s>',
 '<s>',
 'printer',
 'driver',
 'install',
 'fails',
 'with',
 'error',
 '1603',
 '</s>']

## 3) Build vocabulary and handle OOV with <UNK>


In [5]:

# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

freq = Counter(train_tokens_flat)

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = 3
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)


(['mobile', 'app', 'notifications', 'delayed'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>'])

Increasing the min_count raises the OOV threshold, that causes more rare words to be replaced by <UNK>.
This reduces the vocabulary size and sparsity, which can lower perplexity on small datasets by avoiding unseen n-grams.
If the min_count is too high, excessive <UNK> replaces the useful lexical information, increasing perplexity again.
So, there is a trade-off between vocab coverage and model generalization.

## 4) Train n-gram counts (unigram, bigram, trigram)


We will compute:
- `ngram_counts[(w1,...,wn)]`
- `context_counts[(w1,...,w_{n-1})]`

Then probability:
\ndefault:  P(w_n | context) = count(context + w_n) / count(context)

This fails when an n-gram is unseen, so we add smoothing.


In [6]:
def get_ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def train_ngram_counts(texts: List[str], n: int, vocab: set) -> Dict[Tuple[str, ...], int]:
    ngrams_counts = Counter()
    context_counts = Counter()
    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        for ng in get_ngrams(toks, n):
            ngrams_counts[ng] += 1
            context = ng[:-1]
            context_counts[context] += 1

    return ngrams_counts, context_counts


In [7]:
uni_counts, uni_ctx = train_ngram_counts(train_texts, n=1, vocab=vocab)
uni_counts

Counter({('<UNK>',): 74, ('</s>',): 15, ('not',): 4, ('after',): 3})

In [8]:
bi_counts, bi_ctx = train_ngram_counts(train_texts, n=2, vocab=vocab)
bi_counts

Counter({('<UNK>', '<UNK>'): 52,
         ('<s>', '<UNK>'): 15,
         ('<UNK>', '</s>'): 15,
         ('<UNK>', 'not'): 4,
         ('not', '<UNK>'): 4,
         ('<UNK>', 'after'): 3,
         ('after', '<UNK>'): 3})

In [9]:
tri_counts, tri_ctx = train_ngram_counts(train_texts, n=3, vocab=vocab)
tri_counts

Counter({('<UNK>', '<UNK>', '<UNK>'): 33,
         ('<s>', '<s>', '<UNK>'): 15,
         ('<s>', '<UNK>', '<UNK>'): 14,
         ('<UNK>', '<UNK>', '</s>'): 14,
         ('<UNK>', 'not', '<UNK>'): 4,
         ('<UNK>', '<UNK>', 'not'): 3,
         ('<UNK>', 'after', '<UNK>'): 3,
         ('after', '<UNK>', '<UNK>'): 3,
         ('<UNK>', '<UNK>', 'after'): 2,
         ('not', '<UNK>', '<UNK>'): 2,
         ('not', '<UNK>', '</s>'): 1,
         ('<s>', '<UNK>', 'not'): 1,
         ('not', '<UNK>', 'after'): 1})

In [10]:
# Unigram counts (for backoff)

from collections import Counter

uni_counts = Counter()

for sentence in train_texts:
    tokens = tokenize(sentence)
    tokens = [t if t in vocab else "<UNK>" for t in tokens]
    tokens = add_boundaries(tokens, n=3)
    uni_counts.update(tokens)

# Unigram context is empty tuple
uni_ctx = Counter()
uni_ctx[()] = sum(uni_counts.values())


## 5) Add-k smoothing and probability function


Add-k smoothing (a common baseline):
\na) Add *k* to every possible next word count  
b) Normalize by context_count + k * |V|

P_k(w|h) = (count(h,w) + k) / (count(h) + k*|V|)

Where V is the vocabulary.


In [11]:
def prob_addk(ngram: Tuple[str, ...], ngram_counts: Counter, context_counts: Counter, V: int, k: float = 0.5) -> float :

    """Compute addk P(w_n | w_1 ... w_{n-1})
    where ngram = (w_1, w_2, ..., w_n)
    0 < k < = 1
    
    """

    context = ngram[:-1]
    return (ngram_counts[ngram] + k)/(context_counts[context] + k*V)



In [12]:
V = len(vocab)
example = ("<s>", "login")
prob_addk(example, bi_counts, bi_ctx, V, k=0.5)


0.02857142857142857

In [13]:
def trigram_addk_prob(context, word, tri_counts, tri_ctx, V, k=0.5):
    """
    Trigram add-k probability: P(word | w1, w2)
    context = (w1, w2)
    """
    trigram = (context[0], context[1], word)
    return prob_addk(trigram, tri_counts, tri_ctx, V, k=k)


## Backoff Implementation

In [14]:
def backoff_prob(context2, word, k=0.5):
    """
    Backoff language model probability:
    - If trigram context (w1,w2) seen -> use trigram
    - else fallback to bigram using (w2)
    - else fallback to unigram

    context2: tuple of last 2 tokens (w1, w2)
    """
    V = len(vocab)

    # --- 1) Try trigram ---
    if tri_ctx.get(context2, 0) > 0:
        return trigram_addk_prob(context2, word, tri_counts, tri_ctx, V, k=k)

    # --- 2) Backoff to bigram (use last token only) ---
    w2 = context2[-1]
    context1 = (w2,)
    if bi_ctx.get(context1, 0) > 0:
        bigram = (w2, word)
        return prob_addk(bigram, bi_counts, bi_ctx, V, k=k)

    # --- 3) Backoff to unigram ---
    unigram = (word,)
    return prob_addk(unigram, uni_counts, uni_ctx, V, k=k)


## 6) Evaluate: cross-entropy and perplexity on test set


We evaluate an LM by how well it predicts held-out text.

Cross-entropy (average negative log probability):
H = - (1/N) * sum log2 P(w_i | context)

Perplexity:
PP = 2^H

Lower perplexity is better.


In [15]:
def cross_entropy_and_perplexity(test_texts, k=1.0):
    total_log_prob = 0.0
    total_tokens = 0

    V = len(vocab)  # vocab size (includes <UNK>, <s>, </s>)

    for sentence in test_texts:
        toks = tokenize(sentence)
        toks = replace_oov(toks, vocab)          # IMPORTANT: consistent OOV handling
        toks = add_boundaries(toks, n=3)         # trigram model => 2 start tokens

        for i in range(2, len(toks)):
            context = (toks[i-2], toks[i-1])
            word = toks[i]

            p = backoff_prob(context, word, k=k)

            total_log_prob += math.log2(p)
            total_tokens += 1

    H = -total_log_prob / total_tokens
    PP = 2 ** H
    return H, PP


H, PP = cross_entropy_and_perplexity(test_texts, k=1.0)
print(f"Cross-Entropy: {H:.4f}")
print(f"Perplexity: {PP:.4f}")

Cross-Entropy: 0.9262
Perplexity: 1.9002


## 7) Next-word prediction (top-k)


Given a context, compute the probability of each candidate next token and return the top-k.

This mirrors:
- autocomplete in constrained domains
- template suggestion systems
- command prediction in runbooks


In [16]:
def predict_next_word(context_words, top_k=5, smoothing_k=1.0):
    """
    context_words: tuple like ("password", "reset")
    returns: list of (word, prob) sorted high->low
    """
    V = len(vocab)

    w1, w2 = context_words
    w1 = w1.lower()
    w2 = w2.lower()

    # Make context consistent with training preprocessing
    w1, w2 = replace_oov([w1, w2], vocab)
    context = (w1, w2)

    candidates = []
    for word in vocab:
        if word == "<s>":     # never predict start token
            continue

        p = trigram_addk_prob(context, word, tri_counts, tri_ctx, V, k=smoothing_k)
        candidates.append((word, p))

    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[:top_k]


# Example
print("Top-5 next words for context ('password','reset'):")
for w, p in predict_next_word(("password", "reset"), top_k=5, smoothing_k=1.0):
    print(f"{w:>12s}  {p:.4f}")


Top-5 next words for context ('password','reset'):
       <UNK>  0.5965
        </s>  0.2632
         not  0.0702
       after  0.0526


## 8) Simple generation (bigram or trigram)


Text generation is not the main goal in SLMs, but it helps you verify:
- boundary handling
- smoothing
- OOV decisions

We will sample tokens until we hit `</s>`.


In [17]:

def sample_next(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5):
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    words = [w for w in vocab if w != "<s>"]
    probs = []
    for w in words:
        ng = context + (w,)
        probs.append(prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth))
    # Normalize
    s = sum(probs)
    probs = [p/s for p in probs]
    return random.choices(words, weights=probs, k=1)[0]

def generate(n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, max_len: int = 20, k_smooth: float = 0.5):
    tokens = ["<s>"]*(n-1) if n > 1 else []
    out = []
    for _ in range(max_len):
        w = sample_next(tokens, n, ngram_counts, context_counts, vocab, k_smooth=k_smooth)
        if w == "</s>":
            break
        out.append(w)
        tokens.append(w)
    return " ".join(out)

for _ in range(5):
    print("BIGRAM: ", generate(2, bi_counts, bi_ctx, vocab, max_len=18, k_smooth=0.5))
    print("TRIGRAM:", generate(3, tri_counts, tri_ctx, vocab, max_len=18, k_smooth=0.5))
    print()

BIGRAM:  <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
TRIGRAM: <UNK> <UNK>

BIGRAM:  <UNK>
TRIGRAM: <UNK> <UNK> <UNK>

BIGRAM:  <UNK> <UNK>
TRIGRAM: <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>

BIGRAM:  <UNK> <UNK> <UNK> not after after <UNK> <UNK> <UNK>
TRIGRAM: <UNK> <UNK> <UNK> <UNK>

BIGRAM:  <UNK> <UNK> <UNK> <UNK> <UNK> after <UNK> <UNK> <UNK> <UNK>
TRIGRAM: <UNK> <UNK> <UNK>



## 9) Model comparison: effect of n and smoothing


Try different `k` values. Notes:
- `k=1.0` is Laplace smoothing (often too strong)
- smaller `k` (like 0.1 to 0.5) is often better

In real corpora, trigrams often beat bigrams, but require more data.


In [20]:
k_values = [1.0, 0.5, 0.1]

print("Trigram LM comparison (different add-k values)\n")
for k in k_values:
    H, PP = cross_entropy_and_perplexity(test_texts, k=k)
    print(f"k = {k:<4} | Cross-Entropy = {H:.4f} | Perplexity = {PP:.4f}")


Trigram LM comparison (different add-k values)

k = 1.0  | Cross-Entropy = 0.9262 | Perplexity = 1.9002
k = 0.5  | Cross-Entropy = 0.8461 | Perplexity = 1.7976
k = 0.1  | Cross-Entropy = 0.7646 | Perplexity = 1.6989


In [21]:
def top_k_next_words_from_phrase(phrase: str, top_k: int = 5, smoothing_k: float = 1.0):
    """
    Returns top-k next-word predictions for a phrase like: "user cannot"
    Uses trigram if possible, otherwise falls back to bigram.
    Assumes these already exist in your notebook:
      tokenize, replace_oov, prob_addk, trigram_addk_prob,
      vocab, bi_counts, bi_ctx, tri_counts, tri_ctx
    """

    # 1) tokenize the phrase the SAME way as training
    tokens = tokenize(phrase)

    if len(tokens) == 0:
        raise ValueError("Phrase is empty after tokenization.")

    # 2) Decide which model we can use based on how many tokens we have
    V = len(vocab)
    candidates = []

    # --- TRIGRAM (needs 2 context words) ---
    if len(tokens) >= 2:
        w1, w2 = tokens[-2].lower(), tokens[-1].lower()

        # Make context consistent with training OOV handling
        w1, w2 = replace_oov([w1, w2], vocab)
        context = (w1, w2)

        for word in vocab:
            if word == "<s>":   # don't predict start token
                continue
            p = trigram_addk_prob(context, word, tri_counts, tri_ctx, V, k=smoothing_k)
            candidates.append((word, p))

    # --- BIGRAM fallback (only 1 context word) ---
    else:
        w1 = tokens[-1].lower()
        w1 = replace_oov([w1], vocab)[0]
        context = (w1,)

        for word in vocab:
            if word == "<s>":
                continue
            bigram = context + (word,)
            p = prob_addk(bigram, bi_counts, bi_ctx, V, k=smoothing_k)
            candidates.append((word, p))

    # 3) Sort high -> low and return top-k
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[:top_k]


# ===== Example (what the exercise asks) =====
print("Top-5 next words for phrase: 'user cannot'")
for w, p in top_k_next_words_from_phrase("user cannot", top_k=5, smoothing_k=1.0):
    print(f"{w:>12s}  {p:.6f}")


Top-5 next words for phrase: 'user cannot'
       <UNK>  0.596491
        </s>  0.263158
         not  0.070175
       after  0.052632


Although the function requests top-5 predictions, only 4 tokens are returned because the start token "s" is explicitly excluded and the remaining vocabulary size is smaller than 5.

## Exercises (do these during lab)
1) Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.  
2) Change `min_count` (OOV threshold) and explain how perplexity changes.  
3) Implement **backoff**: if a trigram is unseen, fall back to bigram; if unseen, fall back to unigram.  
4) Create a function that returns **top-5 next words** given a phrase like: `"user cannot"`.
